In [1]:
# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from imblearn.over_sampling import SMOTE

In [2]:
# 1. Obtener un set de datos.
df = pd.read_csv("data.csv")

# 2. Buscar valores vacios
print("Missing values:")
print(df.isna().sum())

# 3. Eliminar columna que no agrega valor al entrenamiento
df = df.drop('Employee_ID', axis=1)

# Mostrar distribución de clases
print("\nDistribución original de clases:")
print(df['Attrition'].value_counts(normalize=True) * 100)


Missing values:
Employee_ID                      0
Age                              0
Gender                           0
Marital_Status                   0
Department                       0
Job_Role                         0
Job_Level                        0
Monthly_Income                   0
Hourly_Rate                      0
Years_at_Company                 0
Years_in_Current_Role            0
Years_Since_Last_Promotion       0
Work_Life_Balance                0
Job_Satisfaction                 0
Performance_Rating               0
Training_Hours_Last_Year         0
Overtime                         0
Project_Count                    0
Average_Hours_Worked_Per_Week    0
Absenteeism                      0
Work_Environment_Satisfaction    0
Relationship_with_Manager        0
Job_Involvement                  0
Distance_From_Home               0
Number_of_Companies_Worked       0
Attrition                        0
dtype: int64

Distribución original de clases:
Attrition
No     80.03
Yes 

In [3]:
# Guardar las columnas categóricas
categorical_cols = ['Gender', 'Marital_Status', 'Department', 'Job_Role', 'Overtime']

# Guardar las columnas numéricas
numerical_cols = [col for col in df.columns if col not in categorical_cols + ['Attrition']]

In [4]:
# 4. Separar features y target
X = df.drop('Attrition', axis=1)
y = df['Attrition']

In [5]:
# 5. Dividir en train y test con un split 80 - 20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
# 6. Aplicar las técnicas de escalamiento y transformación de datos
# Transformar datos categóricos mediante One Hot Encoder
encoder = OneHotEncoder(sparse_output=False)
X_train_cat_encoded = encoder.fit_transform(X_train[categorical_cols])
X_test_cat_encoded = encoder.transform(X_test[categorical_cols])

# Escalamiento de datos numéricos mediante StandarScaler
scaler = StandardScaler()
X_train_num_scaled = scaler.fit_transform(X_train[numerical_cols])
X_test_num_scaled = scaler.transform(X_test[numerical_cols])

In [7]:
# 7. Obtener nombres de features
encoded_feature_names = encoder.get_feature_names_out(categorical_cols)

# Crear DataFrames para datos categóricos transformados
X_train_cat_df = pd.DataFrame(X_train_cat_encoded, columns=encoded_feature_names)
X_test_cat_df = pd.DataFrame(X_test_cat_encoded, columns=encoded_feature_names)

# Crear DataFrames para datos numéricos transformados
X_train_num_df = pd.DataFrame(X_train_num_scaled, columns=numerical_cols)
X_test_num_df = pd.DataFrame(X_test_num_scaled, columns=numerical_cols)

In [8]:
# 8. Combinar datos categóricos y numéricos
X_train_processed = pd.concat([X_train_num_df, X_train_cat_df], axis=1)
X_test_processed = pd.concat([X_test_num_df, X_test_cat_df], axis=1)

In [9]:
# 9. Transformar target a valor numerico mediante LabelEncoder
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [10]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_processed, y_train_encoded)

In [11]:
# 8. Implementación de Modelo 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

input_shape = X_train_resampled.shape[1:]

model = Sequential([
    Flatten(input_shape=input_shape),  
    Dense(256, activation="relu"),  
    Dense(1, activation="sigmoid")   
])

/Users/sebastianflores/miniconda3/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [12]:
# 9. Compilamos modelo
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [13]:
# 10. Entrenamiento de modelo
history = model.fit(
    X_train_resampled, y_train_resampled,
    epochs=100,
    validation_data=(X_test_processed, y_test_encoded)
)

Epoch 1/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5313 - loss: 0.6938 - val_accuracy: 0.5107 - val_loss: 0.7082
Epoch 2/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 0s 910us/step - accuracy: 0.6589 - loss: 0.6383 - val_accuracy: 0.6007 - val_loss: 0.6600
Epoch 3/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 0s 876us/step - accuracy: 0.7051 - loss: 0.5962 - val_accuracy: 0.6380 - val_loss: 0.6383
Epoch 4/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 0s 896us/step - accuracy: 0.7404 - loss: 0.5567 - val_accuracy: 0.5393 - val_loss: 0.7232
Epoch 5/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 0s 952us/step - accuracy: 0.7556 - loss: 0.5301 - val_accuracy: 0.6120 - val_loss: 0.6765
Epoch 6/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7769 - loss: 0.4934 - val_accuracy: 0.5917 - val_loss: 0.7024
Epoch 7/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7999 - loss: 0.4663 - val_accuracy: 0.6317 - val_loss: 0.6842
Epoch 8/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8054 - loss: 0.4510 - 

In [14]:
# 11. Evaluación inicial del modelo  
test_loss, test_acc = model.evaluate(X_test_processed, y_test_encoded)
print(f'Accuracy: {test_acc:.4f}')

94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 690us/step - accuracy: 0.7085 - loss: 1.5852
Accuracy: 0.7033


In [15]:
predictions = model.predict(X_test_processed)
classes_x = (predictions > 0.5).astype(int).flatten()

classes_x

94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 722us/step


array([0, 0, 0, ..., 1, 0, 0])

In [16]:
TP = 0
TN = 0
FP = 0
FN = 0

for i in range(len(classes_x)):
  if classes_x[i] == 1:
    if y_test_encoded[i] == 1:
      TP = TP + 1
    else :
      FP = FP + 1
  else:
    if y_test_encoded[i] == 0:
      TN = TN + 1
    else :
      FN = FN + 1


print('         ', 'label neg ', ' label pos')
print('pred neg    ', TN, "        ", FN)
print('pred pos    ', FP, "        ", TP)

precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1 = precision*recall / (precision + recall)
print("Precision:", precision)
print("Recall:", recall)
print("F1:", f1)

          label neg   label pos
pred neg     2005          520
pred pos     370          105
Precision: 0.22105263157894736
Recall: 0.168
F1: 0.09545454545454547
